In [10]:
import time
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
import os
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler

# Configurações gerais
plt.rcParams.update({'figure.max_open_warning': 0})

# Diretório para salvar as figuras
output_dir = 'figuras_lstm_var'
os.makedirs(output_dir, exist_ok=True)

# Atualizar os hiperparâmetros
hyperparams = {
    'model_path': 'modelo_lstm_var_hibrido.keras',
    'num_coefs': 20,
    'sequence_length': 50,
    'lstm_units': 64,
    'dropout_rate': 0.2,
    'l2_regularization': 0.0001,
    'learning_rate': 0.001,
    'epochs': 100,
    'batch_size': 64,
    'early_stopping_patience': 10
}

# Medir o tempo total do script
total_start_time = time.time()

# Medir o tempo de carregamento dos dados
start_time = time.time()

# Carregar os arquivos .mat fornecidos
file_path_temp_coefs = "temp_coefs.mat"
file_path_spatial_modes = "spatial_modes.mat"
file_path_mean_flow = "mean_flow.mat"
file_path_parameters = "parameters.mat"

# Carregar os coeficientes temporais
mat_data_temp = sio.loadmat(file_path_temp_coefs)
ts = mat_data_temp['ts'].flatten()  # Vetor de tempo
coefs = mat_data_temp['coefs']      # Matriz de coeficientes complexos

# Carregar os modos espaciais
mat_data_spatial = sio.loadmat(file_path_spatial_modes)
phi = mat_data_spatial['phi']       # Matriz de modos espaciais

# Carregar o vetor y e a média Xavg
mat_data_mean_flow = sio.loadmat(file_path_mean_flow)
y_positions = mat_data_mean_flow['y'].flatten()  # Vetor de posições espaciais y
Xavg = mat_data_mean_flow['Xavg'].flatten()      # Média do fluxo

# Carregar os modos usados no POD
mat_data_parameters = sio.loadmat(file_path_parameters)
nmodos_pod = mat_data_parameters['nmodes']       # Modos usados no POD

# Medir o tempo de carregamento dos dados
end_time = time.time()
print(f"Tempo de carregamento dos dados: {end_time - start_time:.2f} segundos")

# Definir o número de coeficientes a serem utilizados
num_coefs = hyperparams['num_coefs']
print(f"Número total de modos utilizados: {num_coefs}")

# Separar os coeficientes em parte real e imaginária
real_coefs = np.real(coefs[:num_coefs, :])
imag_coefs = np.imag(coefs[:num_coefs, :])

# Concatenar as partes real e imaginária
coefs_combined = np.vstack([real_coefs, imag_coefs])  # Dimensão (num_coefs*2, N)

# Transpor para ter a forma (N, num_coefs*2)
coefs_combined = coefs_combined.T

# Definir o limite do primeiro intervalo
first_interval_end = 4801  # Posição correspondente ao tempo 1300

# Subamostragem para taxa de 0.5 (pegando a cada 2 pontos)
train_data = coefs_combined[:first_interval_end:2]
val_data = coefs_combined[first_interval_end:]

# Normalizar os dados de treinamento e validação
start_time = time.time()

scalers = []
train_data_normalized = np.zeros_like(train_data)
val_data_normalized = np.zeros_like(val_data)

num_total_coefs = num_coefs * 2

for i in range(num_total_coefs):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    train_data_normalized[:, i] = scaler.fit_transform(train_data[:, i].reshape(-1, 1)).flatten()
    val_data_normalized[:, i] = scaler.transform(val_data[:, i].reshape(-1, 1)).flatten()
    scalers.append(scaler)

end_time = time.time()
print(f"Tempo de normalização dos dados: {end_time - start_time:.2f} segundos")

# Preparar os dados para a LSTM
start_time = time.time()

def create_sequences(input_data, seq_length):
    X = []
    y = []
    num_samples = len(input_data) - seq_length
    for i in range(num_samples):
        X_seq = input_data[i:i+seq_length]
        y_seq = input_data[i+seq_length]
        X.append(X_seq)
        y.append(y_seq)
    X = np.array(X)
    y = np.array(y)
    return X, y

sequence_length = hyperparams['sequence_length']
X_train_lstm, y_train_lstm = create_sequences(train_data_normalized, sequence_length)
X_val_lstm, y_val_lstm = create_sequences(val_data_normalized, sequence_length)

end_time = time.time()
print(f"Tempo de preparação das sequências: {end_time - start_time:.2f} segundos")

# Verificar se o modelo já foi salvo anteriormente
retrain_model = True
model_path = hyperparams['model_path']
if os.path.exists(model_path):
    print("Carregando o modelo salvo...")
    model = load_model(model_path)
    if model.input_shape[1:] != (X_train_lstm.shape[1], X_train_lstm.shape[2]):
        print("O modelo salvo não é compatível com os dados atuais. Treinando um novo modelo.")
        os.remove(model_path)
        retrain_model = True
    else:
        retrain_model = False
else:
    print("Treinando um novo modelo...")
    retrain_model = True

if retrain_model:
    start_time = time.time()

    # Definir o modelo LSTM
    lstm_input = Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]))
    x = LSTM(hyperparams['lstm_units'], activation='tanh',
             kernel_regularizer=l2(hyperparams['l2_regularization']),
             recurrent_dropout=hyperparams['dropout_rate'])(lstm_input)
    x = Dropout(hyperparams['dropout_rate'])(x)
    output = Dense(num_total_coefs, activation='linear')(x)

    # Definir e compilar o modelo
    model = Model(inputs=lstm_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=hyperparams['learning_rate']), loss='mean_squared_error')

    # Definir callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=hyperparams['early_stopping_patience'],
                                   restore_best_weights=True)
    checkpoint = ModelCheckpoint('best_model_lstm.keras', monitor='val_loss', save_best_only=True, mode='min')

    # Treinar o modelo
    history = model.fit(X_train_lstm, y_train_lstm,
                        epochs=hyperparams['epochs'],
                        batch_size=hyperparams['batch_size'],
                        validation_data=(X_val_lstm, y_val_lstm),
                        callbacks=[early_stopping, checkpoint],
                        verbose=1)

    # Salvar o modelo treinado
    model.save(model_path)

    end_time = time.time()
    print(f"Tempo de treinamento do modelo LSTM: {end_time - start_time:.2f} segundos")
else:
    print("Usando o modelo salvo para fazer previsões.")

# Medir o tempo de previsão da LSTM
start_time = time.time()
lstm_predictions_normalized = model.predict(X_val_lstm)
end_time = time.time()
print(f"Tempo de previsão do modelo LSTM: {end_time - start_time:.2f} segundos")

# Reverter a normalização das previsões da LSTM
start_time = time.time()
lstm_predictions = np.zeros_like(lstm_predictions_normalized)
for i in range(num_total_coefs):
    scaler = scalers[i]
    lstm_predictions[:, i] = scaler.inverse_transform(lstm_predictions_normalized[:, i].reshape(-1, 1)).flatten()
end_time = time.time()
print(f"Tempo para reverter a normalização: {end_time - start_time:.2f} segundos")

# Calcular os resíduos entre os valores reais e as previsões da LSTM
actual_values = val_data[sequence_length:]
residuals = actual_values - lstm_predictions

# Ajustar o modelo VAR nos resíduos
start_time = time.time()
model_var = VAR(residuals)
# Selecionar automaticamente a ordem do modelo (número de defasagens) usando o critério AIC
results_aic = model_var.select_order(maxlags=15)
selected_lag = results_aic.aic
print(f"Ordem do modelo VAR nos resíduos selecionada (AIC): {selected_lag}")
model_var_fitted = model_var.fit(selected_lag)
end_time = time.time()
print(f"Tempo de treinamento do modelo VAR: {end_time - start_time:.2f} segundos")

# Previsões do VAR nos resíduos
start_time = time.time()
var_residuals_pred = model_var_fitted.fittedvalues
# Preencher o início com zeros para alinhamento
var_residuals_pred = np.vstack([np.zeros((selected_lag, num_total_coefs)), var_residuals_pred])
end_time = time.time()
print(f"Tempo de previsão do modelo VAR: {end_time - start_time:.2f} segundos")

# Previsão final = Previsão LSTM + Previsão VAR nos resíduos
final_predictions = lstm_predictions[selected_lag:] + var_residuals_pred[selected_lag:]

# Ajustar o tamanho das previsões finais para corresponder aos valores reais
min_length = min(final_predictions.shape[0], actual_values.shape[0])
final_predictions = final_predictions[:min_length]
actual_values = actual_values[selected_lag:][:min_length]

# Separar as partes real e imaginária das previsões e valores reais
pred_real = final_predictions[:, :num_coefs]
pred_imag = final_predictions[:, num_coefs:]
actual_real = actual_values[:, :num_coefs]
actual_imag = actual_values[:, num_coefs:]

# Reconstruir os coeficientes complexos preditos e reais
predicted_coefs = pred_real + 1j * pred_imag
actual_coefs = actual_real + 1j * actual_imag

# Reconstruir X a partir dos coeficientes preditos e dos modos espaciais
phi_reduced = phi[:, :num_coefs]  # Dimensão (387, num_coefs)
Xavg = Xavg.flatten()  # Dimensão (387,)

X_rec_list = []
X_actual_list = []

for i in range(predicted_coefs.shape[0]):
    # Reconstrução predita
    X_rec = Xavg + phi_reduced @ predicted_coefs[i]
    X_rec_list.append(X_rec)
    # Reconstrução real
    X_actual = Xavg + phi_reduced @ actual_coefs[i]
    X_actual_list.append(X_actual)

# Converter as listas em arrays e transpor
X_rec_array = np.array(X_rec_list).T        # Forma (387, num_samples)
X_actual_array = np.array(X_actual_list).T  # Forma (387, num_samples)

# Número de pontos em y
ny = y_positions.shape[0]  # Deve ser 129

# Plotar a comparação entre os perfis originais e reconstruídos de |u|
step = 10  # Número de subplots (ajuste conforme necessário)

plt.figure(figsize=(16, 12))
plt.clf()
for i in range(step):
    plt.subplot(4, 5, i + 1)
    # Índice do tempo
    idx = i * (X_rec_array.shape[1] // step)
    plt.plot(np.abs(X_actual_array[:ny, idx]), y_positions, label='Original')
    plt.plot(np.abs(X_rec_array[:ny, idx]), y_positions, label='Reconstruído', linestyle='--')
    plt.xlabel('|u|')
    plt.ylabel('y')
    plt.title(f'Amostra {idx}')
    plt.grid(True)
    if i == 0:
        plt.legend()
plt.suptitle('Comparação de |u| Original e Reconstruído (Validação) - LSTM + VAR')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(os.path.join(output_dir, 'comparacao_u_original_reconstruido_lstm_var.png'), dpi=400)
plt.close()

# Calcular as métricas de erro separadamente para as partes real e imaginária
mse_real = mean_squared_error(actual_real.flatten(), pred_real.flatten())
mae_real = mean_absolute_error(actual_real.flatten(), pred_real.flatten())
r2_real = r2_score(actual_real.flatten(), pred_real.flatten())

mse_imag = mean_squared_error(actual_imag.flatten(), pred_imag.flatten())
mae_imag = mean_absolute_error(actual_imag.flatten(), pred_imag.flatten())
r2_imag = r2_score(actual_imag.flatten(), pred_imag.flatten())

print(f"Métricas para a parte real (LSTM + VAR):")
print(f"MSE: {mse_real:.6f}")
print(f"MAE: {mae_real:.6f}")
print(f"R²: {r2_real:.6f}")

print(f"Métricas para a parte imaginária (LSTM + VAR):")
print(f"MSE: {mse_imag:.6f}")
print(f"MAE: {mae_imag:.6f}")
print(f"R²: {r2_imag:.6f}")

# Função para calcular NMSE por coeficiente
def calculate_normalized_mse(actual, predicted):
    num_coefs = actual.shape[1]
    nmse = np.zeros(num_coefs)
    for c in range(num_coefs):
        mse = mean_squared_error(actual[:, c], predicted[:, c])
        variance = np.var(actual[:, c])
        # Evitar divisão por zero
        if variance != 0:
            nmse[c] = mse / variance
        else:
            nmse[c] = np.nan  # ou trate conforme apropriado
    return nmse

# Calcular NMSE para a parte real
nmse_real = calculate_normalized_mse(actual_real, pred_real)

# Calcular NMSE para a parte imaginária
nmse_imag = calculate_normalized_mse(actual_imag, pred_imag)

# Calcular NMSE para o módulo dos coeficientes
actual_modulus = np.sqrt(actual_real**2 + actual_imag**2)
predicted_modulus = np.sqrt(pred_real**2 + pred_imag**2)
nmse_modulus = calculate_normalized_mse(actual_modulus, predicted_modulus)

# Calcular a média do NMSE sobre todos os coeficientes
average_nmse_real = np.nanmean(nmse_real)
average_nmse_imag = np.nanmean(nmse_imag)
average_nmse_modulus = np.nanmean(nmse_modulus)

print(f"Média do NMSE para a parte real (LSTM + VAR): {average_nmse_real:.6f}")
print(f"Média do NMSE para a parte imaginária (LSTM + VAR): {average_nmse_imag:.6f}")
print(f"Média do NMSE para o módulo dos coeficientes (LSTM + VAR): {average_nmse_modulus:.6f}")

# Plotar NMSE do Módulo por coeficiente
coefficients = np.arange(1, num_coefs + 1)

plt.figure(figsize=(12, 6))
plt.bar(coefficients, nmse_modulus, width=0.6, label='Módulo (LSTM + VAR)', color='orange')
plt.xlabel('Índice do Coeficiente')
plt.ylabel('NMSE')
plt.title('NMSE por Coeficiente (Módulo) - LSTM + VAR')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'nmse_por_coeficiente_modulo_lstm_var.png'), dpi=400)
plt.close()

# Plotar a função de perda (loss) ao longo das épocas
if retrain_model:
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Perda de Treinamento')
    plt.plot(history.history['val_loss'], label='Perda de Validação')
    plt.title('Função de Perda ao Longo das Épocas - LSTM')
    plt.xlabel('Épocas')
    plt.ylabel('Perda (MSE)')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'funcao_perda_lstm_var.png'), dpi=400)
    plt.close()

# Plotar NMSE para a parte real, imaginária e módulo dos coeficientes
coefficients = np.arange(1, num_coefs + 1)

plt.figure(figsize=(14, 6))

# NMSE da Parte Real
plt.subplot(1, 3, 1)
plt.bar(coefficients, nmse_real, width=0.6, color='blue')
plt.xlabel('Índice do Coeficiente')
plt.ylabel('NMSE')
plt.title('NMSE por Coeficiente - Parte Real (LSTM + VAR)')
plt.grid(True)

# NMSE da Parte Imaginária
plt.subplot(1, 3, 2)
plt.bar(coefficients, nmse_imag, width=0.6, color='green')
plt.xlabel('Índice do Coeficiente')
plt.ylabel('NMSE')
plt.title('NMSE por Coeficiente - Parte Imaginária (LSTM + VAR)')
plt.grid(True)

# NMSE do Módulo
plt.subplot(1, 3, 3)
plt.bar(coefficients, nmse_modulus, width=0.6, color='orange')
plt.xlabel('Índice do Coeficiente')
plt.ylabel('NMSE')
plt.title('NMSE por Coeficiente - Módulo (LSTM + VAR)')
plt.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'nmse_por_coeficiente_todas_lstm_var.png'), dpi=400)
plt.close()

# Adicionar a plotagem da evolução do NMSE ao longo do tempo
nmse_time = np.mean((actual_values - final_predictions)**2, axis=1) / np.var(actual_values, axis=0).mean()

plt.figure(figsize=(12, 6))
plt.plot(nmse_time, label='NMSE ao Longo do Tempo')
plt.xlabel('Amostra de Tempo')
plt.ylabel('NMSE')
plt.title('Evolução do NMSE ao Longo das Amostras de Tempo - LSTM + VAR')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'nmse_tempo_lstm_var.png'), dpi=400)
plt.close()

# Plotar a comparação dos perfis de velocidade 'v' e 'w' originais e reconstruídos

# Exemplo para 'v'
plt.figure(figsize=(16, 12))
plt.clf()
for i in range(step):
    plt.subplot(4, 5, i + 1)
    idx = i * (X_rec_array.shape[1] // step)
    plt.plot(np.abs(X_actual_array[ny:2*ny, idx]), y_positions, label='Original')
    plt.plot(np.abs(X_rec_array[ny:2*ny, idx]), y_positions, label='Reconstruído', linestyle='--')
    plt.xlabel('|v|')
    plt.ylabel('y')
    plt.title(f'Amostra {idx}')
    plt.grid(True)
    if i == 0:
        plt.legend()
plt.suptitle('Comparação de |v| Original e Reconstruído (Validação) - LSTM + VAR')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(os.path.join(output_dir, 'comparacao_v_original_reconstruido_lstm_var.png'), dpi=400)
plt.close()

# Exemplo para 'w'
plt.figure(figsize=(16, 12))
plt.clf()
for i in range(step):
    plt.subplot(4, 5, i + 1)
    idx = i * (X_rec_array.shape[1] // step)
    plt.plot(np.abs(X_actual_array[2*ny:, idx]), y_positions, label='Original')
    plt.plot(np.abs(X_rec_array[2*ny:, idx]), y_positions, label='Reconstruído', linestyle='--')
    plt.xlabel('|w|')
    plt.ylabel('y')
    plt.title(f'Amostra {idx}')
    plt.grid(True)
    if i == 0:
        plt.legend()
plt.suptitle('Comparação de |w| Original e Reconstruído (Validação) - LSTM + VAR')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(os.path.join(output_dir, 'comparacao_w_original_reconstruido_lstm_var.png'), dpi=400)
plt.close()

# Plotar coeficientes aleatórios em subplots
num_plots = 4  # Número de coeficientes a serem plotados
random_indices = np.random.choice(num_coefs, num_plots, replace=False)

plt.figure(figsize=(15, 10))
for idx, coef_index in enumerate(random_indices):
    plt.subplot(2, 2, idx + 1)
    plt.plot(actual_real[:, coef_index], label=f'Parte Real - Coeficiente {coef_index+1} (Real)')
    plt.plot(pred_real[:, coef_index], label=f'Parte Real - Coeficiente {coef_index+1} (Predito)', linestyle='--')
    plt.title(f'Comparação da Parte Real do Coeficiente {coef_index+1} (Validação) - LSTM + VAR')
    plt.xlabel('Amostra de Tempo')
    plt.ylabel('Valor')
    plt.legend()
    plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'comparacao_parte_real_coeficientes_lstm_var.png'), dpi=400)
plt.close()

plt.figure(figsize=(15, 10))
for idx, coef_index in enumerate(random_indices):
    plt.subplot(2, 2, idx + 1)
    plt.plot(actual_imag[:, coef_index], label=f'Parte Imaginária - Coeficiente {coef_index+1} (Real)')
    plt.plot(pred_imag[:, coef_index], label=f'Parte Imaginária - Coeficiente {coef_index+1} (Predito)', linestyle='--')
    plt.title(f'Comparação da Parte Imaginária do Coeficiente {coef_index+1} (Validação) - LSTM + VAR')
    plt.xlabel('Amostra de Tempo')
    plt.ylabel('Valor')
    plt.legend()
    plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'comparacao_parte_imaginaria_coeficientes_lstm_var.png'), dpi=400)
plt.close()

# Exibir o tempo total de execução do script
total_end_time = time.time()
print(f"Tempo total de execução do script: {total_end_time - total_start_time:.2f} segundos")


Tempo de carregamento dos dados: 0.13 segundos
Número total de modos utilizados: 20
Tempo de normalização dos dados: 0.02 segundos
Tempo de preparação das sequências: 0.06 segundos
Treinando um novo modelo...
Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 97ms/step - loss: 0.1030 - val_loss: 0.0805
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0784 - val_loss: 0.0650
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 0.0629 - val_loss: 0.0531
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.0528 - val_loss: 0.0446
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.0451 - val_loss: 0.0382
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0396 - val_loss: 0.0334
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 0.0363 - val_loss: 0.0296
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.0329 - val_loss: 0.0270
Epoch 9/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0315 - val_loss: 0.0249
Epoch 10/100
37/3